In [1]:
import pywt
import os
from pylab import *
from numpy import*
import pandas as pd

In [12]:
df = pd.read_csv('train/train_meta.csv')

In [13]:
df.head()

,ecg_id,patient_id,age,sex,height,weight,nurse,site,device,recording_date,...,static_noise,burst_noise,electrodes_problems,extra_beats,pacemaker,strat_fold,filename_lr,filename_hr,record_name,group
0,9,18792.0,55.0,0,NaN,70.0,2.0,0.0,CS-12 E,1984-12-08 09:44:43,...,", I-AVR,",NaN,NaN,NaN,NaN,10,records100/00000/00009_lr,records500/00000/00009_hr,00009_hr,1
1,34,13619.0,56.0,0,NaN,NaN,2.0,0.0,CS-12 E,1985-02-10 12:04:03,...,", alles,",NaN,NaN,NaN,NaN,9,records100/00000/00034_lr,records500/00000/00034_hr,00034_hr,3
2,43,11315.0,25.0,1,NaN,63.0,2.0,0.0,CS-12 E,1985-03-14 09:38:53,...,NaN,NaN,NaN,NaN,NaN,8,records100/00000/00043_lr,records500/00000/00043_hr,00043_hr,3
3,52,18153.0,35.0,0,NaN,82.0,2.0,0.0,CS-12 E,1985-05-18 08:15:54,...,NaN,NaN,NaN,NaN,NaN,1,records100/00000/00052_lr,records500/00000/00052_hr,00052_hr,2
4,57,16063.0,26.0,0,NaN,93.0,2.0,0.0,CS-12 E,1985-06-06 11:32:43,...,NaN,NaN,NaN,NaN,NaN,10,records100/00000/00057_lr,records500/00000/00057_hr,00057_hr,3


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2101 entries, 0 to 2100
Data columns (total 30 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   ecg_id                        2101 non-null   int64  
 1   patient_id                    2101 non-null   float64
 2   age                           2101 non-null   float64
 3   sex                           2101 non-null   int64  
 4   height                        689 non-null    float64
 5   weight                        905 non-null    float64
 6   nurse                         1947 non-null   float64
 7   site                          2100 non-null   float64
 8   device                        2101 non-null   object 
 9   recording_date                2101 non-null   object 
 10  report                        2101 non-null   object 
 11  scp_codes                     2101 non-null   object 
 12  heart_axis                    1231 non-null   object 
 13  inf

In [15]:
gts = pd.read_csv('train/train_gts.csv')

In [16]:
myocard_list = list(gts['myocard'])
df.insert(loc=len(df.columns), column='myocard', value=myocard_list)

In [17]:
new_df = pd.get_dummies(df, columns=["sex"]) #разделение sex на male и female
new_df.rename(columns = {'sex_1':'female','sex_0':'male'}, inplace = True )

df = pd.concat([new_df],axis=1)

In [18]:
df[['weight', 'height']] = new_df[['weight', 'height']]. fillna(new_df[['weight', 'height']].mean())
#заполненение NaN средними значениями веса и роста

df['scp_codes'] = df.groupby('scp_codes')['myocard'].transform('mean') #стандарт для обмена цифровыми ЭКГ

df['infarction_stadium1'] = df.groupby('infarction_stadium1')['myocard'].transform('mean') #стадия инфаркта
df['infarction_stadium2'] = df.groupby('infarction_stadium2')['myocard'].transform('mean') #вторая стадия инфаркта
df[['infarction_stadium1', 'infarction_stadium2']] = df[['infarction_stadium1', 'infarction_stadium2']].fillna(0)

df[['static_noise', 'burst_noise']] = df[['static_noise', 'burst_noise']].fillna(0) #заполнение пустых значений шума нулём

df['heart_axis'] = df.groupby('heart_axis')['myocard'].transform('mean') #электрическая ось сердца
df['heart_axis'] = df['heart_axis'].fillna(df['heart_axis'].mean())

df['device'] = df.groupby('device')['myocard'].transform('mean') #устройство записи
df['report'] = df.groupby('report')['myocard'].transform('mean') #краткий доклад о экг пациента

df['baseline_drift'] = df.groupby('baseline_drift')['myocard'].transform('mean') #присутствует смещение базовой линии или скачок
df['extra_beats'] = df.groupby('extra_beats')['myocard'].transform('mean') #дополнительные удары(обычно никак не влияют на здоров/болен)
df[['baseline_drift', 'extra_beats']] = df[['baseline_drift', 'extra_beats']].fillna(0)

oft_value1 = df['nurse'].value_counts().idxmax()
df['nurse'] = df['nurse'].fillna(oft_value1) #заполнение NaN наиболее встречающимся номером медсестры

oft_value2 = df['validated_by'].value_counts().idxmax()
df['validated_by'] = df['validated_by'].fillna(oft_value2) #заполнение NaN наиболее встречающимся номером утверждающего кардиолога

df['electrodes_problems'] = df.groupby('electrodes_problems')['myocard'].transform('mean') #проблемы с электрокодами
df['pacemaker'] = df.groupby('pacemaker')['myocard'].transform('mean') #кардиостимулятор(регулирует функцию электрической проводящей системы сердца)
df[['electrodes_problems', 'pacemaker']] = df[['electrodes_problems', 'pacemaker']].fillna(0)

df['static_noise'] = df.groupby('static_noise')['myocard'].transform('mean') #присутствует статистический шум
df['burst_noise'] = df.groupby('burst_noise')['myocard'].transform('mean') #присутствует "взрывной" шум
df[['static_noise', 'burst_noise']] = df[['static_noise', 'burst_noise']].fillna(0)

In [19]:
df.head()

,ecg_id,patient_id,age,height,weight,nurse,site,device,recording_date,report,...,extra_beats,pacemaker,strat_fold,filename_lr,filename_hr,record_name,group,myocard,male,female
0,9,18792.0,55.0,167.343977,70.000000,2.0,0.0,0.196078,1984-12-08 09:44:43,0.227273,...,0.0,0.0,10,records100/00000/00009_lr,records500/00000/00009_hr,00009_hr,1,0,1,0
1,34,13619.0,56.0,167.343977,71.605525,2.0,0.0,0.196078,1985-02-10 12:04:03,0.000000,...,0.0,0.0,9,records100/00000/00034_lr,records500/00000/00034_hr,00034_hr,3,0,1,0
2,43,11315.0,25.0,167.343977,63.000000,2.0,0.0,0.196078,1985-03-14 09:38:53,0.227273,...,0.0,0.0,8,records100/00000/00043_lr,records500/00000/00043_hr,00043_hr,3,0,0,1
3,52,18153.0,35.0,167.343977,82.000000,2.0,0.0,0.196078,1985-05-18 08:15:54,0.000000,...,0.0,0.0,1,records100/00000/00052_lr,records500/00000/00052_hr,00052_hr,2,0,1,0
4,57,16063.0,26.0,167.343977,93.000000,2.0,0.0,0.196078,1985-06-06 11:32:43,0.227273,...,0.0,0.0,10,records100/00000/00057_lr,records500/00000/00057_hr,00057_hr,3,0,1,0


In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2101 entries, 0 to 2100
Data columns (total 32 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   ecg_id                        2101 non-null   int64  
 1   patient_id                    2101 non-null   float64
 2   age                           2101 non-null   float64
 3   height                        2101 non-null   float64
 4   weight                        2101 non-null   float64
 5   nurse                         2101 non-null   float64
 6   site                          2100 non-null   float64
 7   device                        2101 non-null   float64
 8   recording_date                2101 non-null   object 
 9   report                        2101 non-null   float64
 10  scp_codes                     2101 non-null   float64
 11  heart_axis                    2101 non-null   float64
 12  infarction_stadium1           2101 non-null   float64
 13  inf